In [1]:
import pandas as pd

df = pd.DataFrame([{'Name': 'Chris', 'Item Purchased': 'Sponge', 'Cost': 22.50},
                   {'Name': 'Kevyn', 'Item Purchased': 'Kitty Litter', 'Cost': 2.50},
                   {'Name': 'Filip', 'Item Purchased': 'Spoon', 'Cost': 5.00}],
                  index = ['Store 1', 'Store 1', 'Store 2'])

df

,Cost,Item Purchased,Name
Store 1,22.5,Sponge,Chris
Store 1,2.5,Kitty Litter,Kevyn
Store 2,5.0,Spoon,Filip


Pass in a scalar value to add a new column with default value:

In [2]:
df['Delivered'] = True
df

,Cost,Item Purchased,Name,Delivered
Store 1,22.5,Sponge,Chris,True
Store 1,2.5,Kitty Litter,Kevyn,True
Store 2,5.0,Spoon,Filip,True


To assign a different value for every row we could pass in list of values, **which is long enough**, then Pandas will unpack them and assign them to the rows:

In [3]:
df['Feedback'] = ['Positive', None, 'Negative']
df

,Cost,Item Purchased,Name,Delivered,Feedback
Store 1,22.5,Sponge,Chris,True,Positive
Store 1,2.5,Kitty Litter,Kevyn,True,None
Store 2,5.0,Spoon,Filip,True,Negative


If each rows has a unique index, then we could assign the new column identifier to the series. In this approach we can ignore some items:

In [4]:
df.reset_index(inplace=True)
df['Date'] = pd.Series( {0: 'Dec 1', 2: 'mid-May'} )
df

,index,Cost,Item Purchased,Name,Delivered,Feedback,Date
0,Store 1,22.5,Sponge,Chris,True,Positive,Dec 1
1,Store 1,2.5,Kitty Litter,Kevyn,True,None,NaN
2,Store 2,5.0,Spoon,Filip,True,Negative,mid-May


# Merging DataFrames

![Operations on Sets](_files/Operations_on_Sets.png)

Create two DataFrames with some overlap. Both are indexed along the value we want to merge them on - "Name": 

In [5]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR'},
                         {'Name': 'Sally', 'Role': 'Course liasion'},
                         {'Name': 'James', 'Role': 'Grader'}])
staff_df.set_index('Name', inplace=True)
staff_df

,Role
Name,
Kelly,Director of HR
Sally,Course liasion
James,Grader


In [6]:
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business'},
                           {'Name': 'Mike', 'School': 'Law'},
                           {'Name': 'Sally', 'School': 'Engineering'}])
student_df.set_index('Name', inplace=True)
student_df

,School
Name,
James,Business
Mike,Law
Sally,Engineering


`outer`: Use **Union** of keys from both frames, similar to a SQL **Full Outer Join**, sort keys lexicographically:

In [7]:
pd.merge(staff_df, student_df,
         how='outer', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Kelly,Director of HR,NaN
Mike,NaN,Law
Sally,Course liasion,Engineering


`inner`: Use **Intersection** of keys from both frames, similar to a SQL **Inner Join**, preserve the order of the left keys:

In [8]:
pd.merge(staff_df, student_df,
         how='inner', left_index=True, right_index=True)

,Role,School
Name,,
Sally,Course liasion,Engineering
James,Grader,Business


The other common use cases when merging DataFrames - **set Addition** (Left or Right Join).

`left`: Use only keys from left frame, similar to a SQL **Left Join**, preserve key order:

In [9]:
pd.merge(staff_df, student_df,
         how='left', left_index=True, right_index=True)

,Role,School
Name,,
Kelly,Director of HR,NaN
Sally,Course liasion,Engineering
James,Grader,Business


`right`: Use only keys from right frame, similar to a SQL **Right Join**, preserve key order:

In [10]:
pd.merge(staff_df, student_df,
         how='right', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Mike,NaN,Law
Sally,Course liasion,Engineering


We can use columns instead of indices to join on:

In [11]:
staff_df.reset_index(inplace=True)
student_df.reset_index(inplace=True)

pd.merge(staff_df, student_df,
         how='left', left_on='Name', right_on='Name')

,Name,Role,School
0,Kelly,Director of HR,NaN
1,Sally,Course liasion,Engineering
2,James,Grader,Business


Also we can use index and column together:

In [12]:
products_df = pd.DataFrame([{'Product ID': '111', 'Price': '5'},
                            {'Product ID': '112', 'Price': '1'},
                            {'Product ID': '113', 'Price': '7'}])
products_df.set_index('Product ID', inplace=True)

invoices_df = pd.DataFrame([{'Product ID': '111', 'Customer': 'Bob'},
                            {'Product ID': '112', 'Customer': 'Kate'},
                            {'Product ID': '113', 'Customer': 'James'}])

pd.merge(products_df, invoices_df,
         how='outer', left_index=True, right_on='Product ID')

,Price,Customer,Product ID
0,5,Bob,111
1,1,Kate,112
2,7,James,113


**Conflicts between the DataFrames**

The "Director of HR" is on "State Street", while the two students are on "Washington Avenue". But for the student DataFrame, the location information is actually their home address.

The `merge` function preserves this information, but appends an "\_x" or "\_y" to help differentiate between which index went with which column of data. The "\_x" is always the left DataFrame information, and the "\_y" is always the right DataFrame information. And you could control the names of "\_x" and "\_y" with additional parameters if you want to:

In [13]:
staff_df = pd.DataFrame([
    {'Name': 'Kelly', 'Role': 'Director of HR', 'Location': 'State Street'},
    {'Name': 'Sally', 'Role': 'Course liasion', 'Location': 'Washington Avenue'},
    {'Name': 'James', 'Role': 'Grader', 'Location': 'Washington Avenue'}
])

student_df = pd.DataFrame([
    {'Name': 'James', 'School': 'Business', 'Location': '1024 Billiard Avenue'},
    {'Name': 'Mike', 'School': 'Law', 'Location': 'Fraternity House #22'},
    {'Name': 'Sally', 'School': 'Engineering', 'Location': '512 Wilson Crescent'}
])

pd.merge(staff_df, student_df,
         how='left', left_on='Name', right_on='Name')

,Location_x,Name,Role,Location_y,School
0,State Street,Kelly,Director of HR,NaN,NaN
1,Washington Avenue,Sally,Course liasion,512 Wilson Crescent,Engineering
2,Washington Avenue,James,Grader,1024 Billiard Avenue,Business


**Multi-indexing and multiple columns**

For example the first name for students and staff might overlap, but the last name might not. In this case, we use a list of the multiple columns that should be used to join keys on the `left_on` and `right_on` parameters:

In [14]:
staff_df = pd.DataFrame([
    {'First Name': 'Kelly', 'Last Name': 'Desjardins', 'Role': 'Director of HR'},
    {'First Name': 'Sally', 'Last Name': 'Brooks', 'Role': 'Course liasion'},
    {'First Name': 'James', 'Last Name': 'Wilde', 'Role': 'Grader'}
])

student_df = pd.DataFrame([
    {'First Name': 'James', 'Last Name': 'Hammond', 'School': 'Business'},
    {'First Name': 'Mike', 'Last Name': 'Smith', 'School': 'Law'},
    {'First Name': 'Sally', 'Last Name': 'Brooks', 'School': 'Engineering'}
])

In [15]:
staff_df

,First Name,Last Name,Role
0,Kelly,Desjardins,Director of HR
1,Sally,Brooks,Course liasion
2,James,Wilde,Grader


In [16]:
student_df

,First Name,Last Name,School
0,James,Hammond,Business
1,Mike,Smith,Law
2,Sally,Brooks,Engineering


In [17]:
pd.merge(staff_df, student_df,
         how='inner',
         left_on=['First Name','Last Name'],
         right_on=['First Name','Last Name'])

,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering


# Idiomatic Pandas

**Chain Indexing** - `df.loc["Abc"]["Dfg"]` - it's generally a bad practice, Pandas could return a copy of a view depending upon the underlying NumPy library. If we see a `][` we should think about what we are doing.

In [18]:
df = pd.read_csv('_files/census.csv')
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


**Method chaining** - a little bit different case. The general idea behind method chaining is that every method on an object **returns a reference** to that object. The beauty of this is that we can condense many different operations on a DataFrame into one line or at least one statement of code.

The "pandorable" way to write the code with method chaining:

In [19]:
(df.where(df['SUMLEV'] == 50)
    .dropna()
    .set_index(['STNAME', 'CTYNAME'])
    .rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'})).head()

SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME  CTYNAME                                                   
Alabama Autauga County    50.0     3.0       6.0    1.0     1.0   
        Baldwin County    50.0     3.0       6.0    1.0     3.0   
        Barbour County    50.0     3.0       6.0    1.0     5.0   
        Bibb County       50.0     3.0       6.0    1.0     7.0   
        Blount County     50.0     3.0       6.0    1.0     9.0   

                        CENSUS2010POP  Estimates Base 2010  POPESTIMATE2010  \
STNAME  CTYNAME                                                               
Alabama Autauga County        54571.0              54571.0          54660.0   
        Baldwin County       182265.0             182265.0         183193.0   
        Barbour County        27457.0              27457.0          27341.0   
        Bibb County           22915.0              22919.0          22861.0   
        Blount County         57322.0              57322.0          57373.0   

                        POPESTIMATE2011  POPESTIMATE2012     ...       \
STNAME  CTYNAME                                              ...        
Alabama Autauga County          55253.0          55175.0     ...        
        Baldwin County         186659.0         190396.0     ...        
        Barbour County          27226.0          27159.0     ...        
        Bibb County             22733.0          22642.0     ...        
        Blount County           57711.0          57776.0     ...        

                        RDOMESTICMIG2011  RDOMESTICMIG2012  RDOMESTICMIG2013  \
STNAME  CTYNAME                                                                
Alabama Autauga County          7.242091         -2.915927         -3.012349   
        Baldwin County         14.832960         17.647293         21.845705   
        Barbour County         -4.728132         -2.500690         -7.056824   
        Bibb County            -5.527043         -5.068871         -6.201001   
        Blount County           1.807375         -1.177622         -1.748766   

                        RDOMESTICMIG2014  RDOMESTICMIG2015  RNETMIG2011  \
STNAME  CTYNAME                                                           
Alabama Autauga County          2.265971         -2.530799     7.606016   
        Baldwin County         19.243287         17.197872    15.844176   
        Barbour County         -3.904217        -10.543299    -4.874741   
        Bibb County            -0.177537          0.177258    -5.088389   
        Blount County          -2.062535         -1.369970     1.859511   

                        RNETMIG2012  RNETMIG2013  RNETMIG2014  RNETMIG2015  
STNAME  CTYNAME                                                             
Alabama Autauga County    -2.626146    -2.722002     2.592270    -2.187333  
        Baldwin County    18.559627    22.727626    20.317142    18.293499  
        Barbour County    -2.758113    -7.167664    -3.978583   -10.543299  
        Bibb County       -4.363636    -5.403729     0.754533     1.107861  
        Blount County     -0.848580    -1.402476    -1.577232    -0.884411  

[5 rows x 98 columns]

A more "traditional" way of writing code:

In [20]:
df = df[df['SUMLEV']==50]
df.set_index(['STNAME', 'CTYNAME'], inplace=True)
df.rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'}, inplace=True)
df.head()

SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME  CTYNAME                                                   
Alabama Autauga County      50       3         6      1       1   
        Baldwin County      50       3         6      1       3   
        Barbour County      50       3         6      1       5   
        Bibb County         50       3         6      1       7   
        Blount County       50       3         6      1       9   

                        CENSUS2010POP  Estimates Base 2010  POPESTIMATE2010  \
STNAME  CTYNAME                                                               
Alabama Autauga County          54571                54571            54660   
        Baldwin County         182265               182265           183193   
        Barbour County          27457                27457            27341   
        Bibb County             22915                22919            22861   
        Blount County           57322                57322            57373   

                        POPESTIMATE2011  POPESTIMATE2012     ...       \
STNAME  CTYNAME                                              ...        
Alabama Autauga County            55253            55175     ...        
        Baldwin County           186659           190396     ...        
        Barbour County            27226            27159     ...        
        Bibb County               22733            22642     ...        
        Blount County             57711            57776     ...        

                        RDOMESTICMIG2011  RDOMESTICMIG2012  RDOMESTICMIG2013  \
STNAME  CTYNAME                                                                
Alabama Autauga County          7.242091         -2.915927         -3.012349   
        Baldwin County         14.832960         17.647293         21.845705   
        Barbour County         -4.728132         -2.500690         -7.056824   
        Bibb County            -5.527043         -5.068871         -6.201001   
        Blount County           1.807375         -1.177622         -1.748766   

                        RDOMESTICMIG2014  RDOMESTICMIG2015  RNETMIG2011  \
STNAME  CTYNAME                                                           
Alabama Autauga County          2.265971         -2.530799     7.606016   
        Baldwin County         19.243287         17.197872    15.844176   
        Barbour County         -3.904217        -10.543299    -4.874741   
        Bibb County            -0.177537          0.177258    -5.088389   
        Blount County          -2.062535         -1.369970     1.859511   

                        RNETMIG2012  RNETMIG2013  RNETMIG2014  RNETMIG2015  
STNAME  CTYNAME                                                             
Alabama Autauga County    -2.626146    -2.722002     2.592270    -2.187333  
        Baldwin County    18.559627    22.727626    20.317142    18.293499  
        Barbour County    -2.758113    -7.167664    -3.978583   -10.543299  
        Bibb County       -4.363636    -5.403729     0.754533     1.107861  
        Blount County     -0.848580    -1.402476    -1.577232    -0.884411  

[5 rows x 98 columns]

`.apply()` - applies function along input axis of DataFrame.

**NOTE:** In the current implementation **apply calls func twice on the first column/row** to decide whether it can take a fast or slow code path. This can lead to unexpected behavior if func has side-effects, as they will take effect twice for the first column/row.

In [21]:
import numpy as np

def min_max(row):
    """Finds a min/max values in particular row of data.
    Returns a new row of data.
    """
    # Create slice of a row by projecting the population columns
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    # Use min and max functions and create a new series with a label values
    return pd.Series({'min': np.min(data), 'max': np.max(data)})

`.apply()` takes the function and the axis on which to operate as parameters. Here `axis` is really the parameter of the index to use. To apply across all rows, we pass axis equal to one:

axis : {0 or 'index', 1 or 'columns'}, default 0
* 0 or 'index': apply function to each column
* 1 or 'columns': apply function to each row

In [22]:
df.apply(min_max, axis=1).head(n=10)

max       min
STNAME  CTYNAME                            
Alabama Autauga County    55347.0   54660.0
        Baldwin County   203709.0  183193.0
        Barbour County    27341.0   26489.0
        Bibb County       22861.0   22512.0
        Blount County     57776.0   57373.0
        Bullock County    10887.0   10606.0
        Butler County     20944.0   20154.0
        Calhoun County   118437.0  115620.0
        Chambers County   34153.0   33993.0
        Cherokee County   26084.0   25859.0

We can take the row values and add in new columns indicating the max and minimum scores:

In [23]:
def min_max(row):
    """Finds a min/max values in particular row of data.
    Returns a existing row with a new additional data.
    """
    # Create slice of a row by projecting the population columns
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    # Add new columns to existing row
    row['min'] = np.min(data)
    row['max'] = np.max(data)
    return row

In [24]:
df.apply(min_max, axis=1).head()

SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME  CTYNAME                                                   
Alabama Autauga County    50.0     3.0       6.0    1.0     1.0   
        Baldwin County    50.0     3.0       6.0    1.0     3.0   
        Barbour County    50.0     3.0       6.0    1.0     5.0   
        Bibb County       50.0     3.0       6.0    1.0     7.0   
        Blount County     50.0     3.0       6.0    1.0     9.0   

                        CENSUS2010POP  Estimates Base 2010  POPESTIMATE2010  \
STNAME  CTYNAME                                                               
Alabama Autauga County        54571.0              54571.0          54660.0   
        Baldwin County       182265.0             182265.0         183193.0   
        Barbour County        27457.0              27457.0          27341.0   
        Bibb County           22915.0              22919.0          22861.0   
        Blount County         57322.0              57322.0          57373.0   

                        POPESTIMATE2011  POPESTIMATE2012    ...     \
STNAME  CTYNAME                                             ...      
Alabama Autauga County          55253.0          55175.0    ...      
        Baldwin County         186659.0         190396.0    ...      
        Barbour County          27226.0          27159.0    ...      
        Bibb County             22733.0          22642.0    ...      
        Blount County           57711.0          57776.0    ...      

                        RDOMESTICMIG2013  RDOMESTICMIG2014  RDOMESTICMIG2015  \
STNAME  CTYNAME                                                                
Alabama Autauga County         -3.012349          2.265971         -2.530799   
        Baldwin County         21.845705         19.243287         17.197872   
        Barbour County         -7.056824         -3.904217        -10.543299   
        Bibb County            -6.201001         -0.177537          0.177258   
        Blount County          -1.748766         -2.062535         -1.369970   

                        RNETMIG2011  RNETMIG2012  RNETMIG2013  RNETMIG2014  \
STNAME  CTYNAME                                                              
Alabama Autauga County     7.606016    -2.626146    -2.722002     2.592270   
        Baldwin County    15.844176    18.559627    22.727626    20.317142   
        Barbour County    -4.874741    -2.758113    -7.167664    -3.978583   
        Bibb County       -5.088389    -4.363636    -5.403729     0.754533   
        Blount County      1.859511    -0.848580    -1.402476    -1.577232   

                        RNETMIG2015       min       max  
STNAME  CTYNAME                                          
Alabama Autauga County    -2.187333   54660.0   55347.0  
        Baldwin County    18.293499  183193.0  203709.0  
        Barbour County   -10.543299   26489.0   27341.0  
        Bibb County        1.107861   22512.0   22861.0  
        Blount County     -0.884411   57373.0   57776.0  

[5 rows x 100 columns]

Example of how you might calculate the max of the columns using the apply and lambda:

In [25]:
cols = ['POPESTIMATE2010',
        'POPESTIMATE2011',
        'POPESTIMATE2012',
        'POPESTIMATE2013',
        'POPESTIMATE2014',
        'POPESTIMATE2015']

df.apply(lambda x: np.max(x[cols]), axis=1).head()

STNAME   CTYNAME       
Alabama  Autauga County     55347.0
         Baldwin County    203709.0
         Barbour County     27341.0
         Bibb County        22861.0
         Blount County      57776.0
dtype: float64

# Group by

In [26]:
df = pd.read_csv('_files/census.csv')
df = df[df['SUMLEV']==50]
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411
